<a href="https://colab.research.google.com/github/Felipanjos/a3_ia_2022.2/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### D. Preparação dos dados

In [122]:
import pandas as pd
import numpy as np
from math import ceil, floor
import random
import string
from itertools import groupby


In [99]:
df = pd.read_csv('imdb_top_1000.csv')

In [100]:
df = df[['Series_Title', 'Released_Year', 'Runtime', 'Genre', 'IMDB_Rating', 'Meta_score', 'Director', 'No_of_Votes', 'Gross']]
df

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Meta_score,Director,No_of_Votes,Gross
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80.0,Frank Darabont,2343110,"28,341,469"
1,The Godfather,1972,175 min,"Crime, Drama",9.2,100.0,Francis Ford Coppola,1620367,"134,966,411"
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84.0,Christopher Nolan,2303232,"534,858,444"
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,90.0,Francis Ford Coppola,1129952,"57,300,000"
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,96.0,Sidney Lumet,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...
995,Breakfast at Tiffany's,1961,115 min,"Comedy, Drama, Romance",7.6,76.0,Blake Edwards,166544,NaN
996,Giant,1956,201 min,"Drama, Western",7.6,84.0,George Stevens,34075,NaN
997,From Here to Eternity,1953,118 min,"Drama, Romance, War",7.6,85.0,Fred Zinnemann,43374,"30,500,000"
998,Lifeboat,1944,97 min,"Drama, War",7.6,78.0,Alfred Hitchcock,26471,NaN


##### a) Selecionando 10% das colunas

In [12]:
qtd_colunas = len(df.columns)
qtd_colunas

9

In [13]:
p_colunas = round(qtd_colunas * 10 / 100)
p_colunas

1

No dataset utilizado existem somente 9 colunas, dessa forma:

*    10% de 9 = 0.9
*    Aproximando para 1, fica somente uma coluna a ser selecionada como referência para a modificação dos dados



In [14]:
coluna_aleatoria = df.sample(n=p_colunas, axis='columns').keys()[0]
coluna_aleatoria

'Series_Title'

##### a) Selecionando 3% dos dados

In [15]:
p_3 = int(df.shape[0] * 3 / 100)
p_3

30

##### a) Excluindo 3% dos dados aleatoriamente

In [16]:
reg_nulos = df[df[coluna_aleatoria].isnull()]
reg_nulos

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Meta_score,Director,No_of_Votes,Gross


In [17]:
qtd_reg_nulos = reg_nulos.shape[0]
qtd_reg_nulos

0

In [18]:
reg_excluidos = df.sample(n=p_3)
reg_exc_index = reg_excluidos.index.array
reg_exc_index


<PandasArray>
[773, 550, 274,  24, 561, 139,  84, 361, 624, 603, 598, 677, 784, 414, 538,
 323,  62, 203, 641, 529, 433, 951,  99, 794, 593,  42, 435, 407, 104, 675]
Length: 30, dtype: int64

In [22]:
df.loc[reg_exc_index, coluna_aleatoria] = np.nan

In [24]:
df.loc[773]

Series_Title                        NaN
Released_Year                      1960
Runtime                         144 min
Genre                    Drama, Mystery
IMDB_Rating                         7.9
Meta_score                          NaN
Director         Michelangelo Antonioni
No_of_Votes                       26542
Gross                               NaN
Name: 550, dtype: object

##### a) Alterando 3% dos dados aleatoriamente

In [42]:
def gerar_string_aleatoria():
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(10)) 

In [43]:
def gerar_numero_aleatorio():
    return random.randit(0, 1000)

In [44]:
df_selecao_nao_nulo = df[~df[coluna_aleatoria].isnull()]

In [45]:
reg_alterados = df_selecao_nao_nulo.sample(n=p_3)

In [64]:
reg_alter_index = reg_alterados.index.array
reg_alter_index

<PandasArray>
[931, 568, 640, 972, 149, 191, 536, 622, 699, 423, 805, 778, 815, 541, 711,
 396, 403, 782,  87, 576, 144, 302, 255, 131,  15, 370, 949, 259, 587, 125]
Length: 30, dtype: int64

In [58]:
colunas_numericas = df.describe().columns
if coluna_aleatoria in colunas_numericas:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_numero_aleatorio()
else:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_string_aleatoria()


#### b) Limpeza dos dados

i. Codificação One-Hot

In [131]:

teste = list(df['Genre'].unique())
lista_generos = []
for item in teste:
    lista_generos.extend(item.split(', '))
lista_generos = np.unique(np.array(lista_generos))

array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'], dtype='<U9')